In [ ]:
import pandas as pd
import os

from abcd_tools.utils.io import load_tabular
from abcd_tools.utils.ConfigLoader import load_yaml

In [ ]:
raw_path = "../../data/01_raw"
sst_behavioral_path = os.path.join(raw_path, "tabular/core/imaging/mri_y_tfmr_sst_beh.csv")

In [ ]:
params = load_yaml("../parameters.yaml")
list(params)

In [ ]:
def load_sst_behavioral(sst_behavioral_path: str, params: dict) -> pd.DataFrame:
    """Load the SST behavioral data

    Args:
        sst_behavioral_path (str): Path to NDA SST behavioral data
        params (dict): Parameters loaded from YAML file

    Returns:
        pd.DataFrame: renamed and filtered SST behavioral data
    """
    sst_behavioral = load_tabular(sst_behavioral_path,
        cols=params['beh_rt_vars'],
        timepoints=['baseline_year_1_arm_1'])
    return sst_behavioral
sst_behavioral = load_sst_behavioral(sst_behavioral_path, params)

In [ ]:
rdex_map_path = os.path.join(raw_path, "SST_MAPS_base_r3.csv")
pd.read_csv(rdex_map_path)

def load_rdex_map(rdex_map_path: str, tpt='baseline_year_1_arm_1',
    exclude_vars: list=['tf.natural', 'gf.natural']) -> pd.DataFrame:
    """Load RDEX MAP estimates

    Args:
        rdex_map_path (str): Path to exported RDEX MAP estimates
        tpt (str, optional): Impose timepoint. Defaults to 'baseline_year_1_arm_1'.

    Returns:
        pd.DataFrame: RDEX MAP estimates with index set to src_subject_id and eventname
    """
    rdex_map = pd.read_csv(rdex_map_path)
    rdex_map = rdex_map.rename(columns={'subjectkey': 'src_subject_id'})
    rdex_map = rdex_map.drop(columns=exclude_vars)
    rdex_map.insert(1, 'eventname', tpt)

    return rdex_map.set_index(['src_subject_id', 'eventname'])

rdex_map = load_rdex_map(rdex_map_path)

In [ ]:
rdex_map

In [ ]:
def filter_behavioral(rdex_map: pd.DataFrame, 
    sst_behavioral: pd.DataFrame) -> pd.DataFrame:
    """Filter SST behavioral data to subjects with RDEX MAP estimates

    Args:
        rdex_map (pd.DataFrame): RDEX MAP estimates
        sst_behavioral (pd.DataFrame): SST behavioral data

    Returns:
        pd.DataFrame: SST behavioral data for subjects with RDEX MAP estimates
    """

    rdex_joined = rdex_map.join(sst_behavioral, how='inner')

    print(f"Subjects with RDEX MAP estimates: {len(rdex_map)}")
    print(f"{len(rdex_map) - len(rdex_joined)} subjects missing {list(sst_behavioral.columns)}")
    print(f"Subjects with complete behavioral data: {len(rdex_joined)}")

    return rdex_joined


rdex_joined = filter_behavioral(rdex_map, sst_behavioral)  
rdex_joined

In [ ]:
def save_csv(df: pd.DataFrame, path: str):
    df.to_csv(path)

save_csv(rdex_joined, "../../data/03_primary/rdex_joined.csv")

In [ ]:
load_tabular("../../data/03_primary/rdex_joined.csv")

In [ ]:
rdex_map.join(sst_behavioral, how='inner')